## Cleansing

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("new_data.csv")

In [7]:
df.head()

,State/Area,Year,Month,Total Unemployment in State/Area,Total Employment in State/Area,Total Civilian Labor Force in State/Area,Total Civilian Non-Institutional Population in State/Area
0,Alabama,2006,3,"87,830","2,068,505","2,156,335","3,510,348"
1,Alaska,2006,3,"23,561","323,717","347,278","487,558"
2,Arizona,2006,3,"128,852","2,808,195","2,937,047","4,587,109"
3,Arkansas,2006,3,"69,437","1,275,609","1,345,046","2,134,861"
4,California,2006,3,"865,781","16,730,993","17,596,774","26,956,842"


In [9]:
df = df[["State/Area","Year","Month","Total Unemployment in State/Area","Total Employment in State/Area"]]

In [11]:
df.head()

,State/Area,Year,Month,Total Unemployment in State/Area,Total Employment in State/Area
0,Alabama,2006,3,"87,830","2,068,505"
1,Alaska,2006,3,"23,561","323,717"
2,Arizona,2006,3,"128,852","2,808,195"
3,Arkansas,2006,3,"69,437","1,275,609"
4,California,2006,3,"865,781","16,730,993"


In [13]:
# Remove commas and convert to integers
df['Total Unemployment in State/Area'] = df['Total Unemployment in State/Area'].str.replace(',', '').astype(int)
df['Total Employment in State/Area'] = df['Total Employment in State/Area'].str.replace(',', '').astype(int)


In [15]:
df.head()

,State/Area,Year,Month,Total Unemployment in State/Area,Total Employment in State/Area
0,Alabama,2006,3,87830,2068505
1,Alaska,2006,3,23561,323717
2,Arizona,2006,3,128852,2808195
3,Arkansas,2006,3,69437,1275609
4,California,2006,3,865781,16730993


In [29]:

# Rename columns
new_column_names = {
    'State/Area': 'State',
    'Year': 'Year',
    'Month': 'month',
    'Total Unemployment in State/Area': 'Unemployment',
    'Total Employment in State/Area': 'Employment'
}

df.rename(columns=new_column_names, inplace=True)


In [31]:
# Add an ID column with unique, incrementing values starting from 1
df['ID'] = range(1, len(df) + 1)

# Move the ID column to the start of the DataFrame
columns = ['ID'] + [col for col in df.columns if col != 'ID']
df = df[columns]

In [33]:
df.head()

,ID,State,Year,month,Unemployment,Employment
0,1,Alabama,2006,3,87830,2068505
1,2,Alaska,2006,3,23561,323717
2,3,Arizona,2006,3,128852,2808195
3,4,Arkansas,2006,3,69437,1275609
4,5,California,2006,3,865781,16730993


In [35]:
df.to_csv("employment_clean_data.csv",index=False)

## Loading the data now

In [37]:
import psycopg2
import pandas as pd

# Set up the connection details
conn = psycopg2.connect(
    host="localhost",      # PostgreSQL server address
    database="AOL_schema",    # Replace with your database name
    user="postgres",  # Replace with your PostgreSQL username
    password="Waah9233" # Replace with your PostgreSQL password
)

In [39]:
cur = conn.cursor()

In [43]:
# Read data from the CSV file
data = pd.read_csv("employment_clean_data.csv")


In [45]:
data.head()

,ID,State,Year,month,Unemployment,Employment
0,1,Alabama,2006,3,87830,2068505
1,2,Alaska,2006,3,23561,323717
2,3,Arizona,2006,3,128852,2808195
3,4,Arkansas,2006,3,69437,1275609
4,5,California,2006,3,865781,16730993


In [53]:
# Loop through each row in the DataFrame and insert into the ANONDIM table
for index, row in data.iterrows():
    # Convert the types explicitly to native Python types
    
    ID = int(row['ID'])
    state = str(row['State'])
    year = int(row['Year']) 
    month = int(row['month'])
    unemploymnet = int(row['Unemployment'])
    employment = int(row['Employment'])
    
    # Prepare the insert statement
    insert_query = """
    INSERT INTO EMPLOYMENTSTATS (ID,State,Year,month,Unemployment,employment) 
    VALUES (%s, %s,%s,%s,%s,%s)
    """
    
    # Prepare the data tuple
    data_tuple = (ID,state,year,month,unemploymnet,employment)
    
    # Execute the insert statement
    cur.execute(insert_query, data_tuple)

# Commit the transaction
conn.commit()